# Logging

**Sources:**

- [Fast api logging essentials](https://marketsplash.com/fastapi-logging/);

## Set up

In [3]:
from requests import get

The following cell runs a Docker container that utilizes a specified file to build a FastAPI application.

In [32]:
!docker run --rm -itd\
    --name test_container\
    -v ./logging_files/app.py:/app.py\
    -p 8000:8000 \
    fastapi_experiment \
    uvicorn --host 0.0.0.0 --reload app:app

d4b4de203b936f4f61e4a7e4eb255328fd5f2679681909031215ef8df2d5067f


Don't forget to stop the container.

In [31]:
!docker stop test_container

test_container


## View logs

Let's see what the default logs of the fast api look like. We will make an application to write something to the console and then extract it from the docker container.

In the following cell we've just defined a simple fastapi program that throws a `HTTPException` if called with a `False` argument.

In [1]:
%%writefile logging_files/app.py
from fastapi import FastAPI, HTTPException

app = FastAPI()

@app.get("/{is_ok}")
def handle(is_ok : bool):
    if is_ok:
        return "im fine"
    else:
        raise HTTPException(status_code=404, detail="Item not found")

Overwriting logging_files/app.py


Now let's send some requests to the application and view it's log last lines:

In [14]:
get("http://localhost:8000/True")
get("http://localhost:8000/False")
!docker logs --tail 2 test_container

INFO:     172.17.0.1:37404 - "GET /True HTTP/1.1" 200 OK
INFO:     172.17.0.1:37420 - "GET /False HTTP/1.1" 404 Not Found


## Loggers

The following fastapi program returns all available loggers in the fast api.

In [43]:
%%writefile logging_files/app.py
import logging
from fastapi import FastAPI, HTTPException

app = FastAPI()

@app.get("/")
def handle()->list[str]:
    return [name for name in logging.root.manager.loggerDict]

Overwriting logging_files/app.py


So let's check what does we have.

In [44]:
response = get("http://localhost:8000")
response.content

b'["concurrent.futures","concurrent","asyncio","uvicorn.error","uvicorn","uvicorn.access","fastapi"]'